# Cryptarithmetic

An example of cryptarithmetic is the equation

```text
 ODD +
 ODD =
EVEN
```

where each letter represents a digit and where the equation holds true. First consideration: if the numbers are in base 10, ODD must be greater than 500. It cannot be 500 because this would mean O=5, D=0, and 1000 should be XDDD (we don't yet know X). Also D must be > 5, since there clearly is a carry digit. We also realize that E = 1, because ODD must be <= 999 (it cannot be 999, since the digits are different), so the sum must be strictly lower than 2000. So, the first and third digits must be 1. N must be an even number, since it's the unitary part of 2*D. Using some more inferences we can come up with two solutions: 655 + 655 = 1310 or 855 + 855 = 1710.

## Brute Force Solution

One possible approach could be implementing all the rules of arithmetc (carry digits, odd/even). This is a challenging task, even just for addition. We would like to find a shortcut. Another possibility would be trying all possibilities. Given 10 digits there are $10! \approx 3\times10^6$, this seems feasible, although not fast. For each of the letters, we can consider each of the $10!$ permuatations of the digits. We would replace the letters with the values and keep iterating until the equation holds.

## Inventory of Concepts

We are dealing with the following concepts:

1. Equations. Two types: the original ones (with letters), and the "filled-in" (with digits).
2. Letters.
3. Digits.
4. Assignment of a letter to a digit.
5. Evaluation or validation that the equation is correct.

How do we represent these concepts? Norvig suggests this approach:

- The original equation can be expressed as a string.
- The filled-in equation can also be represented as a string.
- The letters are single characters (one-character strings).
- The numbers are also represented as single characters.
- We need a mapping that associates letters to numbers. We can leverage `str.translate()`.
- For the evaluation we can use `eval()`, that takes a string and evaluates it as an expression.

### `eval`

In [1]:
eval('2 + 2')

4

In [2]:
eval('2 + 2 == 3')

False

### Translation Tables

We can create a translation table with the function `str.maketrans()`. If, for example, we want to translate `'ABC'` to `'123'` we can write:

In [3]:
table = str.maketrans('ABC', '123')
f = 'A + B == C'
f.translate(table)

'1 + 2 == 3'

In [4]:
eval(f.translate(table))

True

Let's define a function `valid()` that returns `True` or `False` if the function is valid or not. It should also return `False` if the equation returns an error like `1/0` (remember, we are considering all 4 operators). The one below is my version.

In [5]:
def valid(f):
    "Formula f is valid iff it has no numbers with leading zero and evals true."
    try:
        return eval(f)
    except ZeroDivisionError:
        return False

def test_valid():
    assert valid('1 + 3 == 4') is True
    assert valid('6/2 == 3') is True
    assert valid('1/0') is False

test_valid()

This is Norvig's version, which checks for more general errors, including non-numeric characters. `ArithmeticError` is a superclass of `ZeroDivisionError`.

In [6]:
import re

def valid(f):
    """Formula is valid iff it has no numbers with leading zeros and evals True"""
    try:
        return not re.search(r'\b0[0-9]', f) and eval(f) is True
    except ArithmeticError:
        return False

### Explanation of Norvig's implementation

In the case of `ODD + ODD = EVEN` `EVEN` = 3435 would be a valid number, but `EVEN` = 0435 would not. We want to avoid cases where numbers have a leading zero. This could appear anywhere in a formula. The regular expression `r'\b0[0-9]` looks for a 0 appearing at a *word boundary*, marked by `\b`.

There is a more profound reason for avoiding numbers with leading zeros: in Python they are interpreted as octal, therefore a number like 012 would be interpreted as decimal 10.